In [2]:
from dotenv import load_dotenv
import os

load_dotenv()  # laddar .env-filen
GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")

print(f"GITHUB_TOKEN: {GITHUB_TOKEN[:8] if GITHUB_TOKEN else 'Not found'}...")

GITHUB_TOKEN: ghp_jijf...


In [1]:
import requests
from datetime import datetime, timedelta

# Define repo info
owner = "hm-group"
repo = "cruisecontrol-application-frontend"

# Calculate date 7 days ago in ISO format
since = (datetime.utcnow() - timedelta(days=7)).isoformat() + "Z"

# GitHub API endpoint
url = f"https://api.github.com/repos/{owner}/{repo}/commits"
headers = {"Authorization": f"token {GITHUB_TOKEN}"}
params = {"since": since}

response = requests.get(url, headers=headers, params=params)
commits = response.json()

commit_data = []
for commit in commits:
    sha = commit.get("sha")
    title = commit["commit"]["message"].split('\n')[0]  # First line as title

    # Get the commit details to fetch the diff
    commit_url = commit["url"]
    commit_response = requests.get(commit_url, headers=headers)
    commit_details = commit_response.json()

    # Get file changes (diffs)
    files = commit_details.get("files", [])
    diffs = []
    for f in files:
        filename = f.get("filename")
        patch = f.get("patch")  # Unified diff for the file
        diffs.append({"filename": filename, "diff": patch})
    data = {"sha": sha, "title": title, "diffs": diffs}
    commit_data.append({"sha": sha, "title": title, "diffs": diffs})

print(commit_data)

NameError: name 'GITHUB_TOKEN' is not defined

In [9]:
import json
import sqlite3
import time
# Read first 20 repos from all_repos.json
with open('./all_repos.json', 'r') as f:
    repos = json.load(f)
repos = repos[:20]  # Only first 100

# Setup SQLite DB
conn = sqlite3.connect('commits.db')
c = conn.cursor()
c.execute('''CREATE TABLE IF NOT EXISTS commits (
    repo TEXT,
    sha TEXT,
    title TEXT,
    diffs TEXT
    )''')

# Function to fetch commits for a repo
def fetch_commits(owner, repo):
    since = (datetime.utcnow() - timedelta(days=7)).isoformat() + "Z"
    url = f"https://api.github.com/repos/{owner}/{repo}/commits"
    headers = {"Authorization": f"token {GITHUB_TOKEN}"}
    params = {"since": since}
    response = requests.get(url, headers=headers, params=params)
    commits = response.json()
    commit_data = []
    for commit in commits:
        sha = commit.get("sha")
        title = commit["commit"]["message"].split('\n')[0]
        commit_url = commit["url"]
        commit_response = requests.get(commit_url, headers=headers)
        commit_details = commit_response.json()
        files = commit_details.get("files", [])
        diffs = []
        for f in files:
            filename = f.get("filename")
            patch = f.get("patch")
            diffs.append({"filename": filename, "diff": patch})
        commit_data.append({"sha": sha, "title": title, "diffs": diffs})
        time.sleep(0.2)  # To avoid rate limits
    return commit_data

# Iterate over repos and store commit data
for repo_info in repos:
    repo_url = repo_info.get('url')
    if not repo_url or not repo_url.startswith("https://github.com/"):
        continue
    parts = repo_url.replace("https://github.com/", "").split("/")
    if len(parts) != 2:
        continue
    owner, repo = parts
    print(f"Fetching commits for {owner}/{repo}")
    data = fetch_commits(owner, repo)
    for commit in data:
        c.execute('INSERT INTO commits (repo, sha, title, diffs) VALUES (?, ?, ?, ?)',
                  (repo_url, commit['sha'], commit['title'], json.dumps(commit['diffs'])))
    conn.commit()

conn.close()

Fetching commits for hm-group/commercetools-servicebus
Fetching commits for hm-group/cart
Fetching commits for hm-group/cart
Fetching commits for hm-group/common-cart-cache-refresh
Fetching commits for hm-group/common-cart-cache-refresh
Fetching commits for hm-group/common-cart-asb-scaler
Fetching commits for hm-group/common-cart-asb-scaler
Fetching commits for hm-group/hm-commerce-starter-pack-aggregator
Fetching commits for hm-group/hm-commerce-starter-pack-aggregator
Fetching commits for hm-group/cicd-acceleration-framework
Fetching commits for hm-group/cicd-acceleration-framework
Fetching commits for hm-group/hm-group-github-bot
Fetching commits for hm-group/hm-group-github-bot
Fetching commits for hm-group/hm-csp-backend-commands
Fetching commits for hm-group/hm-csp-backend-commands
Fetching commits for hm-group/hm-csp-config-ct-app
Fetching commits for hm-group/hm-csp-config-ct-app
Fetching commits for hm-group/hm-csp-ct-adyen-integration-extension
Fetching commits for hm-group/h